In [2]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime as dt
from timeit import default_timer as timer
warnings.filterwarnings('ignore')
# from google.colab import drive
# drive.mount('/content/drive/')

# UMTS Impacted and Blocking Cell Identifier

In [5]:
#*****************************************************USER INPUT******************************************************
# Report name (PRS): GUL Interference Analysis...._HAL
# file = '/content/drive/MyDrive/Interference/GUL Interference Analysis for WN.xlsx'
# output = '/content/drive/MyDrive/Interference'

file = 'GUL Interference Analysis for KYN.xlsx'
output = 'Interference//'
#**********************************************************************************************************************

# start_date = input('Start Date (YYYY-MM-DD):')
# end_date = input('End Date (YYYY-MM-DD):')

start_date = '2023-07-23'
end_date = '2023-08-21'

start_date = dt.strptime(start_date, '%Y-%m-%d').date()
end_date = dt.strptime(end_date, '%Y-%m-%d').date()

start = timer()
def replace_sector(data):
    if data == 'D':
        return 'A'
    elif data == 'E':
        return 'B'
    elif data == 'F':
        return 'C'
    else:
        return data


# Loading data
u = pd.read_excel(file, sheet_name='UMTS', na_values=['NIL', '/0'], engine='openpyxl')
u.dropna(inplace=True)

# # Date range
u['Date'] = pd.to_datetime(u['Date'])
date_range = np.arange(start_date, end_date)
u = u[u['Date'].isin(date_range)]

# Necessary columns inserted
u.insert(column='TCP Utilization(%)', loc=9, value=(10**(u['MeanTCP']/10)/1000)/(10**(u['Max Transmit Power of Cell (0.1dBm)']/100)/1000)*100)
u.insert(column='Sector', loc=4, value=u['Cell Name'].str[10:11])
u.insert(column='System', loc=3, value=u['Cell Name'].str[0:2])
u.insert(column='Site', loc=4, value=u['Cell Name'].str[2:10])
u.insert(column='New Sector', loc=4, value=u.apply(lambda x: replace_sector(x['Sector']), axis=1))
u.insert(column='Sector Name', loc=4, value=u['Site'] + '_' + u['New Sector'])

df_gp = u.groupby(['Sector Name','Cell Name', 'Site', 'Sector', 'New Sector', 'System']).agg(
                {'Mean RTWP': np.mean,
                 'TCP Utilization(%)': np.mean,
                 'Cell Availability(%)': np.mean,
             'Mute Call Ratio nom': np.sum}).reset_index()

df_impact = df_gp[(df_gp['Cell Availability(%)'] > 0) & (df_gp['Mean RTWP'] >-95) & (df_gp['System'] == 'UN')].reset_index(drop=True)
df_block = df_gp[(df_gp['Cell Availability(%)'] > 0) & (df_gp['Mean RTWP'] >-90) & (df_gp['System'] == 'UN')].reset_index(drop=True)
sectors = list(df_block['Sector Name'].unique())

df_sec = df_gp[df_gp['Sector Name'].isin(sectors)].reset_index(drop=True)
df_pv = df_sec.pivot_table(index='Sector Name', columns=['System', 'Sector'], values='TCP Utilization(%)', aggfunc='mean')
df_pv['Total'] = df_pv['UN'].sum(axis=1) + df_pv['UT'].sum(axis=1)
propose_sectors = list(df_pv.loc[((df_pv['Total'] <= 80) & (df_pv['UT'].mean(axis=1).notnull()))].index)
propose_sectors = df_gp.loc[df_gp['Sector Name'].isin(propose_sectors), :].reset_index(drop=True)
impact_site_count = len(df_impact['Site'].unique())
block_site_count = len(propose_sectors[(propose_sectors['System'] == 'UN')]['Site'].unique())
summary = (pd.DataFrame({'Impact sites': impact_site_count, 'Propose to block': block_site_count},
                        index=['Site count'])).T
with pd.ExcelWriter(output+'/UMTS Blocking Proposal_HAL.xlsx') as writer:
    summary.to_excel(writer, sheet_name='Summary')
    df_impact.to_excel(writer, index=False, sheet_name='Impacted list')
    propose_sectors.to_excel(writer, index=False, sheet_name='Blocking proposal')
end = timer()
print('Total time usage:', (end-start)/60, 'minutes')
print('Operation completed')

Total time usage: 0.13877795178333752 minutes
Operation completed


# UMTS & LTE Blocking Site Counter

In [8]:
#*****************************************************USER INPUT******************************************************
# Blocking tracker
record = 'Blocking cells list_v1.0 (update).xlsx'
# Propose Cell list
# propose = '/content/drive/MyDrive/Interference/Proposed Cells.xlsx'
# output = '/content/drive/MyDrive/Interference'

propose = 'Proposed Cells.xlsx'
output = 'Interference//'
#**********************************************************************************************************************


summary = pd.DataFrame(data='', columns=['UMTS Site Counts', 'LTE Site Counts'], index=['Old blocked', 'New block'])
rd = pd.read_excel(record, index_col='Sr. No')
pp_u = pd.read_excel(propose, sheet_name='UMTS')
pp_l = pd.read_excel(propose, sheet_name='LTE')

# UMTS
pp_u['UMTS Site Name'] = pp_u['Cell Name'].str[2:10]
pp_u['Type'] = ''
pp_u.loc[(pp_u['UMTS Site Name'].isin(rd[(rd['Techonology'] == 'UMTS')]['Site Name'])),'Type'] = 'Old'
pp_u.loc[(~pp_u['UMTS Site Name'].isin(rd[(rd['Techonology'] == 'UMTS')]['Site Name'])),'Type'] = 'New'

pp_u['UMTS Site Name'].isin(rd[(rd['Techonology'] == 'UMTS')]['Site Name'])
to_block = len(pp_u[pp_u['Type'] == 'New']['UMTS Site Name'].unique())
blocked = len(pp_u[pp_u['Type'] == 'Old']['UMTS Site Name'].unique())
summary.loc[['Old blocked', 'New block'], 'UMTS Site Counts'] = [blocked, to_block]

# LTE
pp_l['LTE Site Name'] = pp_l['Cell Name'].str[2:10]
pp_l['Type'] = ''
pp_l.loc[(pp_l['LTE Site Name'].isin(rd[(rd['Techonology'] == 'LTE')]['Site Name'])),'Type'] = 'Old'
pp_l.loc[(~pp_l['LTE Site Name'].isin(rd[(rd['Techonology'] == 'LTE')]['Site Name'])),'Type'] = 'New'

pp_l['LTE Site Name'].isin(rd[(rd['Techonology'] == 'LTE')]['Site Name'])
to_block = len(pp_l[pp_l['Type'] == 'New']['LTE Site Name'].unique())
blocked = len(pp_l[pp_l['Type'] == 'Old']['LTE Site Name'].unique())
summary.loc[['Old blocked', 'New block'], 'LTE Site Counts'] = [blocked, to_block]

with pd.ExcelWriter(output+'/UMTS and LTE blocking site count_HAL.xlsx') as writer:
    summary.to_excel(writer, sheet_name='Summary')
    pp_u.to_excel(writer, sheet_name='UMTS Cell List', index=False)
    pp_l.to_excel(writer, sheet_name='LTE Cell List', index=False)

print('Operation completed')

Operation completed
